In [83]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [282]:
input_path = "E:\\deeplearning\\Hepatocarcinomes\\TCGA\\"
out_path = input_path + "heatmap\\survival\\"

In [283]:
clusters = pd.read_csv(input_path + 'heatmap\\consensus_sample_clusters_filtered-sangro_median_km_euclidean_2_3.csv', sep=',')
os = pd.read_csv(input_path + 'processed\\Overall_Survival_legacy.txt', sep='\t')
dfs = pd.read_csv(input_path + 'processed\\Disease_Free_Survival_legacy.txt', sep='\t')

In [284]:
print(clusters.shape) # (340, 2)
display(clusters.head(5))
print()
print(os.shape) # (340, 2)
display(os.head(5))
print()
print(dfs.shape) # (340, 2)
display(dfs.head(5))
print()

(340, 2)


,Sample,Cluster
0,AADA.01A,Cluster 01
1,A5SJ.01A,Cluster 01
2,AACF.01A,Cluster 01
3,A5NO.01A,Cluster 01
4,AAE0.01A,Cluster 01



(440, 4)


,Study ID,Patient ID,OS_STATUS,OS_MONTHS
0,lihc_tcga,TCGA-2V-A95S,0:LIVING,NaN
1,lihc_tcga,TCGA-2Y-A9GS,1:DECEASED,23.78
2,lihc_tcga,TCGA-2Y-A9GT,1:DECEASED,53.35
3,lihc_tcga,TCGA-2Y-A9GU,0:LIVING,63.70
4,lihc_tcga,TCGA-2Y-A9GV,1:DECEASED,83.18



(440, 4)


,Study ID,Patient ID,DFS_STATUS,DFS_MONTHS
0,lihc_tcga,TCGA-2V-A95S,0:DiseaseFree,NaN
1,lihc_tcga,TCGA-2Y-A9GS,1:Recurred/Progressed,3.35
2,lihc_tcga,TCGA-2Y-A9GT,1:Recurred/Progressed,35.58
3,lihc_tcga,TCGA-2Y-A9GU,0:DiseaseFree,63.70
4,lihc_tcga,TCGA-2Y-A9GV,1:Recurred/Progressed,71.91


### Extract survival data

In [285]:
def exact_info(df, item):
    i=0
    clusters[item] = ''
    for sample in clusters["Sample"]:
        info = set(df[df['Patient ID'].str.contains(sample[:-4][-4:])][item])
        if len(info) != 1 :
#             raise ValueError('Multiple or no values found.')
            i=i+1
        else:
            info = list(info)[0]
            clusters.loc[clusters["Sample"]==sample, item] = info
    return i # count of samples not found

In [286]:
print(exact_info(df = os, item = 'OS_STATUS')) # 0
print(exact_info(df = os, item = 'OS_MONTHS')) # 0

0
0


In [287]:
print(exact_info(df = dfs, item = 'DFS_STATUS')) # 0
print(exact_info(df = dfs, item = 'DFS_MONTHS')) # 0

0
0


In [288]:
print(clusters.shape) # (340, 6)
display(clusters.head(5))
print()

(340, 6)


,Sample,Cluster,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
0,AADA.01A,Cluster 01,0:LIVING,40.51,0:DiseaseFree,40.51
1,A5SJ.01A,Cluster 01,0:LIVING,22.93,1:Recurred/Progressed,10.12
2,AACF.01A,Cluster 01,1:DECEASED,11.99,1:Recurred/Progressed,3.91
3,A5NO.01A,Cluster 01,0:LIVING,0.66,0:DiseaseFree,0.66
4,AAE0.01A,Cluster 01,0:LIVING,18.23,0:DiseaseFree,18.23


### Preprocess

In [289]:
for i in range(len(clusters)):
    clusters.loc[i,'OS_STATUS'] = clusters.loc[i,'OS_STATUS'][0]
    try:
        clusters.loc[i,'DFS_STATUS'] = clusters.loc[i,'DFS_STATUS'][0]
    except:
        pass

In [290]:
display(clusters.dtypes)
clusters = clusters.astype({'OS_STATUS':'float64', 'OS_MONTHS':'float64', 'DFS_STATUS':'float64', 'DFS_MONTHS':'float64'})
display(clusters.dtypes)

Sample        object
Cluster       object
OS_STATUS     object
OS_MONTHS     object
DFS_STATUS    object
DFS_MONTHS    object
dtype: object

Sample         object
Cluster        object
OS_STATUS     float64
OS_MONTHS     float64
DFS_STATUS    float64
DFS_MONTHS    float64
dtype: object

In [291]:
print(clusters.shape) # (340, 6)
display(clusters.head(5))
print()

(340, 6)


,Sample,Cluster,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
0,AADA.01A,Cluster 01,0.0,40.51,0.0,40.51
1,A5SJ.01A,Cluster 01,0.0,22.93,1.0,10.12
2,AACF.01A,Cluster 01,1.0,11.99,1.0,3.91
3,A5NO.01A,Cluster 01,0.0,0.66,0.0,0.66
4,AAE0.01A,Cluster 01,0.0,18.23,0.0,18.23


### *ATTENTION!*

In [292]:
# ## Merge cluster 01 and cluster 03 manually
# clusters = clusters.replace("Cluster 02", "Cluster 01")

### Explore data

In [293]:
# Misssing values
clusters.groupby(["Cluster"]).count()

,Sample,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
Cluster,,,,,
Cluster 01,127,127,127,111,111
Cluster 02,171,171,171,149,149
Cluster 03,42,42,41,37,36


In [294]:
# Only 1 missing data for OS, with OS_STATUS but no OS_MONTHS
clusters[clusters['OS_MONTHS'].isna()]

,Sample,Cluster,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
304,A95S.01A,Cluster 03,0.0,NaN,0.0,NaN


In [295]:
# Censored values
clusters.groupby(["OS_STATUS"]).count()

,Sample,Cluster,OS_MONTHS,DFS_STATUS,DFS_MONTHS
OS_STATUS,,,,,
0.0,220,220,219,220,219
1.0,120,120,120,77,77


### Version 1: Append the count matrix

In [277]:
# Load and visualize data
counts = pd.read_csv(input_path + 'processed\\count_matrix_filtered_full.csv', sep='\t', index_col=0).T
counts.rename(index={"4073-01B":"X4073-01B", "5259-01A":"X5259-01A", "4072-01B":"X4072-01B","5261-01A":"X5261-01A", 
                     "5262-01A":"X5262-01A","5263-01A":"X5263-01A","5264-01A":"X5264-01A","5258-01A":"X5258-01A"},inplace=True)
print(counts.shape)
display(counts.head(5))

(340, 733)


,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,ANGPT2,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
AADA-01A,78112,17,24,167,341,3605,6592,1517,91,58,...,588,2,263,21,14,376,215,565,402,75
A5SJ-01A,195194,20,85,825,253,4263,28808,3659,511,642,...,382,365,262,9,29,356,3487,953,419,45
AACF-01A,54394,20,27,318,131,6242,9177,26,4,464,...,815,8,334,2,51,88,403,285,193,89
A9GZ-01A,387356,264,20,184,322,9003,4217,3457,132,220,...,1068,436,1533,0,3,43,299,746,175,9
A113-01A,275578,73,64,266,167,4781,24453,2374,130,867,...,677,303,825,9,119,96,485,1173,506,60


In [278]:
# replacement. For example, 'A4NK-01A' to 'A4NK.01A'
for i in range(len(counts.index.values)):
    counts.index.values[i] = counts.index.values[i].replace("-",".")
display(counts.head(5))

,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,ANGPT2,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
AADA.01A,78112,17,24,167,341,3605,6592,1517,91,58,...,588,2,263,21,14,376,215,565,402,75
A5SJ.01A,195194,20,85,825,253,4263,28808,3659,511,642,...,382,365,262,9,29,356,3487,953,419,45
AACF.01A,54394,20,27,318,131,6242,9177,26,4,464,...,815,8,334,2,51,88,403,285,193,89
A9GZ.01A,387356,264,20,184,322,9003,4217,3457,132,220,...,1068,436,1533,0,3,43,299,746,175,9
A113.01A,275578,73,64,266,167,4781,24453,2374,130,867,...,677,303,825,9,119,96,485,1173,506,60


In [279]:
clusters = clusters.join(counts, on="Sample")

In [280]:
# Export data
clusters.to_csv(input_path+"heatmap//data_counts_clusters_survival_filtered-sangro_median_consensus_km_euclidean_3.csv",sep="\t", index=False)

In [281]:
clusters[clusters.Sample=="X5262.01A"] #"X4072.01B" "X5261.01A" "X5262.01A" "X5263.01A" "X5264.01A" "X5258.01A"

,Sample,Cluster,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS,A2M,ACVR1C,ADAM12,ADM,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
106,X5262.01A,Cluster 01,1.0,3.38,NaN,NaN,147759,18,276,740,...,255,672,18,2,12,121,2480,535,537,40


### Version 2: Append the FPKM values

In [296]:
# Load and visualize data
fpkm = pd.read_csv(input_path + 'processed\\fpkm_final_log2-median-centering.csv', sep='\t', index_col=0).T
fpkm.rename(index={"4073-01B":"X4073-01B", "5259-01A":"X5259-01A", "4072-01B":"X4072-01B","5261-01A":"X5261-01A", 
                     "5262-01A":"X5262-01A","5263-01A":"X5263-01A","5264-01A":"X5264-01A","5258-01A":"X5258-01A"},inplace=True)
print(fpkm.shape)
display(fpkm.head(5))

(340, 733)


Gene Name,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,ANGPT2,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
AADA-01A,0.580986,-0.042060,0.013547,-0.734360,0.716873,0.176478,-0.473462,-0.060915,0.120029,-0.449146,...,1.986813,-0.748394,0.791654,0.160138,0.135815,1.214148,-0.239418,0.170838,0.319786,0.819394
A5SJ-01A,0.979374,-0.071784,0.098334,0.301992,-0.105847,-0.428254,0.707962,0.268944,0.756201,0.556600,...,0.911169,0.156321,0.257405,0.027693,0.167687,0.599456,2.378281,0.060643,0.026891,0.039873
AACF-01A,-0.436710,-0.055145,-0.007120,-0.439411,-0.342741,0.445181,-0.495030,-3.887819,-0.340829,0.528913,...,1.964025,-0.724446,0.692565,-0.001063,0.598002,-0.032797,0.049167,-0.642371,-0.133674,0.670390
A9GZ-01A,2.521986,0.630880,-0.022410,-0.878293,0.411041,1.067124,-1.441670,0.723679,0.170189,0.040839,...,2.395214,0.595501,2.513506,-0.013435,-0.185548,-0.227816,-0.164037,0.194186,-0.135175,-0.345550
A113-01A,0.771957,-0.012114,-0.000270,-1.283949,-0.609093,-0.882153,-0.216758,-0.961504,-0.117368,0.398080,...,0.981836,-0.222799,0.806006,0.011926,0.641115,-0.244449,-0.531706,-0.187144,-0.079972,-0.050226


In [297]:
# replacement. For example, 'A4NK-01A' to 'A4NK.01A'
for i in range(len(fpkm.index.values)):
    fpkm.index.values[i] = fpkm.index.values[i].replace("-",".")
display(fpkm.head(5))

Gene Name,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,ANGPT2,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
AADA.01A,0.580986,-0.042060,0.013547,-0.734360,0.716873,0.176478,-0.473462,-0.060915,0.120029,-0.449146,...,1.986813,-0.748394,0.791654,0.160138,0.135815,1.214148,-0.239418,0.170838,0.319786,0.819394
A5SJ.01A,0.979374,-0.071784,0.098334,0.301992,-0.105847,-0.428254,0.707962,0.268944,0.756201,0.556600,...,0.911169,0.156321,0.257405,0.027693,0.167687,0.599456,2.378281,0.060643,0.026891,0.039873
AACF.01A,-0.436710,-0.055145,-0.007120,-0.439411,-0.342741,0.445181,-0.495030,-3.887819,-0.340829,0.528913,...,1.964025,-0.724446,0.692565,-0.001063,0.598002,-0.032797,0.049167,-0.642371,-0.133674,0.670390
A9GZ.01A,2.521986,0.630880,-0.022410,-0.878293,0.411041,1.067124,-1.441670,0.723679,0.170189,0.040839,...,2.395214,0.595501,2.513506,-0.013435,-0.185548,-0.227816,-0.164037,0.194186,-0.135175,-0.345550
A113.01A,0.771957,-0.012114,-0.000270,-1.283949,-0.609093,-0.882153,-0.216758,-0.961504,-0.117368,0.398080,...,0.981836,-0.222799,0.806006,0.011926,0.641115,-0.244449,-0.531706,-0.187144,-0.079972,-0.050226


In [298]:
clusters = clusters.join(fpkm, on="Sample")

In [299]:
# Export data
clusters.to_csv(input_path+"heatmap//data_fpkm_clusters_survival_filtered-sangro_median_consensus_km_euclidean_3.csv",sep="\t", index=False)